# Data Preparation

In [1]:
import pandas as pd
import holidays

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel


In [2]:
# Laden der Daten
# Passe den Pfad entsprechend an, falls die Daten in einem spezifischen Ordner liegen
df = pd.read_csv('../data/raw/sickness_table.csv', parse_dates=['date'])

## 1. Datenbereinigung

In der Datenanalyse wurden folgende Aspekte der Datenqualität überprüft:

- **Fehlende Werte:** Es wurde festgestellt, dass keine fehlenden Werte in den Daten vorliegen. Somit sind keine Imputationsmethoden oder Auffüllstrategien notwendig.
- **Duplikate:** Die Daten wurden auf Duplikate überprüft, und es konnte bestätigt werden, dass keine doppelten Einträge vorhanden sind. Ein Entfernen doppelter Zeilen ist daher nicht erforderlich.
- **Ausreißer:** Während der Analyse wurden mögliche Ausreißer in den numerischen Variablen untersucht. Die vorhandenen Ausreißer scheinen jedoch alle plausibel und nachvollziehbar zu sein (z. B. Spitzenwerte bei Notrufen oder Krankenständen zu bestimmten Zeiten). Daher wurde entschieden, die Ausreißer nicht weiter zu beschneiden oder zu entfernen.

# Forcast der zukünftigen Werte

In [13]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

def forecast_n_sick_calls(df, forecast_days=30):
    # Zeilenweise Prognosen für `n_sick` und `calls` erstellen und zurückgeben
    forecasts = {}
    
    for feature in ['n_sick', 'calls',  'n_duty']:
        series = df[feature]
        
        # ARIMA-Modell (anpassbar für SARIMA, Prophet, etc.)
        model = ARIMA(series, order=(5,1,2))  # Beispielhafte ARIMA-Parameter
        model_fit = model.fit()
        
        # Prognose erstellen
        forecast = model_fit.forecast(steps=forecast_days)
        forecasts[feature] = forecast.values  # Werte als Array speichern
    
    # DataFrame für die prognostizierten Werte
    forecast_df = pd.DataFrame(forecasts)
    forecast_df['date'] = pd.date_range(start=df['date'].max() + pd.Timedelta(days=1), periods=forecast_days, freq='D')
    
    return forecast_df

# Beispiel für die Anwendung der Prognosefunktion
forecast_df = forecast_n_sick_calls(df)
forecast_df


C:\Users\palmierima\PycharmProjects\modelEngineeringCaseStudy2\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\palmierima\PycharmProjects\modelEngineeringCaseStudy2\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\palmierima\PycharmProjects\modelEngineeringCaseStudy2\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\palmierima\PycharmProjects\modelEngineeringCaseStudy2\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible 

,n_sick,calls,n_duty,date
0,75.045053,9369.061436,1900.0,2019-05-28
1,74.606011,9377.074530,1900.0,2019-05-29
2,78.512621,9419.346746,1900.0,2019-05-30
3,77.095856,9431.380945,1900.0,2019-05-31
4,74.605025,9450.345845,1900.0,2019-06-01
5,76.817678,9426.644482,1900.0,2019-06-02
6,78.331633,9449.798352,1900.0,2019-06-03
7,75.380769,9425.835888,1900.0,2019-06-04
8,75.225242,9448.411572,1900.0,2019-06-05
9,78.230000,9426.058741,1900.0,2019-06-06


## 2. Feature Engineering

### 2.1. Feature Enrichment

In [3]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import holidays

# Custom Transformer für kalendarische Features und Feiertage
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, date_column='date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        years = range(X[self.date_column].dt.year.min(), X[self.date_column].dt.year.max() + 1)
        self.de_holidays = holidays.Germany(years=years)
        return self
    
    def transform(self, X):
        X = X.copy()
        X['month'] = X[self.date_column].dt.month.astype('category')
        X['day_of_week'] = X[self.date_column].dt.weekday.astype('category')
        X['week'] = X[self.date_column].dt.isocalendar().week.astype('category')
        X['quarter'] = X[self.date_column].dt.quarter.astype('category')
        X['season'] = (X['month'].astype(int) % 12 // 3 + 1).astype('category')  # 1=Winter, 2=Spring, etc.
        X['holiday'] = X[self.date_column].apply(lambda x: 1 if x in self.de_holidays else 0).astype(bool)
        X['weekend'] = X['day_of_week'].apply(lambda x: 1 if x >= 5 else 0).astype(bool)
        return X

# Custom Transformer für saisonale Durchschnittswerte
class SeasonalAverages(BaseEstimator, TransformerMixin):
    def __init__(self, feature_columns, groupby_columns):
        self.feature_columns = feature_columns
        self.groupby_columns = groupby_columns
    
    def fit(self, X, y=None):
        # Durchschnittswerte für die Ebenen Monat und Kombination aus Wochentag, Woche und Monat
        self.month_avg = X.groupby('month')[self.feature_columns].mean()
        self.combined_avg = X.groupby(['day_of_week', 'week', 'month'])[self.feature_columns].mean()
        return self
    
    def transform(self, X):
        X = X.copy()
        
        # Hinzufügen der monatlichen Durchschnittswerte
        for feature in self.feature_columns:
            X[f'{feature}_avg_month'] = X['month'].map(self.month_avg[feature]).astype(float)
        
        # Erstellen eines Index für die kombinierte Granularität
        X['time_index'] = X[self.groupby_columns].astype(str).agg('_'.join, axis=1)
        combined_avg = self.combined_avg.reset_index()
        combined_avg['time_index'] = combined_avg[self.groupby_columns].astype(str).agg('_'.join, axis=1)
        
        # Hinzufügen der Durchschnittswerte für die kombinierte Granularität
        for feature in self.feature_columns:
            avg_col_name_combined = f'{feature}_avg_combined'
            X = X.merge(combined_avg[['time_index', feature]].rename(columns={feature: avg_col_name_combined}),
                        on='time_index', how='left')
        
        # Entfernen des temporären Indexes
        X.drop(columns=['time_index'], inplace=True)
        return X

# Custom Transformer zum Entfernen unerwünschter Spalten
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)

# Feature-Engineering-Pipeline
feature_engineering_pipeline = Pipeline([
    ('date_features', DateFeatureExtractor(date_column='date')),
    ('seasonal_averages', SeasonalAverages(
        feature_columns=['n_sick', 'calls', 'n_duty'],
        groupby_columns=['day_of_week', 'week', 'month']
    )),
    ('drop_unnecessary_columns', DropColumns(columns_to_drop=['Unnamed: 0', 'n_sby', 'dafted', 'n_sick', 'calls', 'date', 'n_duty']))
])

# Pipeline auf die Daten anwenden
df_transformed = feature_engineering_pipeline.fit_transform(df)

# Ergebnisse anzeigen
df_transformed.head()





C:\Users\palmierima\AppData\Local\Temp\2\ipykernel_25196\162832584.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.month_avg = X.groupby('month')[self.feature_columns].mean()
C:\Users\palmierima\AppData\Local\Temp\2\ipykernel_25196\162832584.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.combined_avg = X.groupby(['day_of_week', 'week', 'month'])[self.feature_columns].mean()


,sby_need,month,day_of_week,week,quarter,season,holiday,weekend,n_sick_avg_month,calls_avg_month,n_duty_avg_month,n_sick_avg_combined,calls_avg_combined,n_duty_avg_combined
0,4.0,4,4,13,2,2,False,False,63.675,8266.4,1825.0,73.00,8154.0,1700.0
1,70.0,4,5,13,2,2,False,True,63.675,8266.4,1825.0,54.50,9150.0,1750.0
2,0.0,4,6,13,2,2,False,True,63.675,8266.4,1825.0,59.00,8080.0,1800.0
3,0.0,4,0,14,2,2,False,False,63.675,8266.4,1825.0,65.75,9676.5,1825.0
4,0.0,4,1,14,2,2,False,False,63.675,8266.4,1825.0,65.25,9132.0,1825.0


# Train-Test-Split

In [4]:
# Setze die Zielvariable
target = 'sby_need'

# Definiere die Merkmale (X) und die Zielvariable (y)
X = df_transformed.drop(columns=[target])
y = df_transformed[target]

# Führe einen zeitlichen Train-Test-Split durch (80 % Training, 20 % Test) mit shuffle=False
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



### 2.2. Scaling und Encoding

In [5]:
# Pipeline für Lineare Regression und SVR (mit Scaling, Encoding und PCA)
preprocessor_lr_svr = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), make_column_selector(dtype_include='category'))
        ], remainder='passthrough')),# Behalte alle übrigen Spalten im Originalzustand
    ('pca', PCA(n_components=0.95)) # Behalte 95 % der Varianz
])

# Pipeline für Tree-basierte Modelle (nur Encoding und Feature Selection, ohne Skalierung)
preprocessor_tree = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', 'passthrough', make_column_selector(dtype_include=['int64', 'float64'])),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), make_column_selector(dtype_include='category'))
        ], remainder='passthrough')),  # Behalte alle übrigen Spalten im Originalzustand
    ('feature_selection', SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold='0.001*mean', max_features=20))
])


## Model Training

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [7]:
# Daten aufteilen und sicherstellen, dass `y` nur für `fit_transform` übergeben wird
X_train_tree_fs = preprocessor_tree.fit_transform(X_train, y_train)  # `y` hier verwenden
X_test_tree_fs = preprocessor_tree.transform(X_test)  # Kein `y` hier verwenden

X_train_lr_pca = preprocessor_lr_svr.fit_transform(X_train)  # `y` für PCA-Pipeline optional
X_test_lr_pca = preprocessor_lr_svr.transform(X_test)  # Kein `y` hier verwenden

X_train_tree_fs

array([[6.36750000e+01, 8.26640000e+03, 1.82500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.36750000e+01, 8.26640000e+03, 1.82500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [6.36750000e+01, 8.26640000e+03, 1.82500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [8.52473118e+01, 7.75541935e+03, 1.80000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.52473118e+01, 7.75541935e+03, 1.80000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.52473118e+01, 7.75541935e+03, 1.80000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Baseline-Modell: Lineare Regression
lr_model = LinearRegression()
lr_model.fit(X_train_lr_pca, y_train)
y_pred_lr = lr_model.predict(X_test_lr_pca)

# Evaluierung der Linearen Regression
print("Lineare Regression - Evaluation:")
print("MSE:", mean_squared_error(y_test, y_pred_lr))
print("MAE:", mean_absolute_error(y_test, y_pred_lr))
print("R^2:", r2_score(y_test, y_pred_lr))

# 2. Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_tree_fs, y_train)
y_pred_rf = rf_model.predict(X_test_tree_fs)

# Evaluierung des Random Forest
print("\nRandom Forest - Evaluation:")
print("MSE:", mean_squared_error(y_test, y_pred_rf))
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("R^2:", r2_score(y_test, y_pred_rf))

# 3. Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train_tree_fs, y_train)
y_pred_gb = gb_model.predict(X_test_tree_fs)

# Evaluierung des Gradient Boosting
print("\nGradient Boosting - Evaluation:")
print("MSE:", mean_squared_error(y_test, y_pred_gb))
print("MAE:", mean_absolute_error(y_test, y_pred_gb))
print("R^2:", r2_score(y_test, y_pred_gb))

# 4. Support Vector Regression
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)  # Standardwerte, können angepasst werden
svr_model.fit(X_train_lr_pca, y_train)  # PCA transformierte Daten verwenden
y_pred_svr = svr_model.predict(X_test_lr_pca)

# Evaluierung der Support Vector Regression
print("\nSupport Vector Regression - Evaluation:")
print("MSE:", mean_squared_error(y_test, y_pred_svr))
print("MAE:", mean_absolute_error(y_test, y_pred_svr))
print("R^2:", r2_score(y_test, y_pred_svr))



Lineare Regression - Evaluation:
MSE: 10834.593657884678
MAE: 60.14573595397706
R^2: 0.04163031242014126

Random Forest - Evaluation:
MSE: 11185.13849953585
MAE: 49.42279165495398
R^2: 0.010623007394771733

Gradient Boosting - Evaluation:
MSE: 10678.689898296363
MAE: 48.91398849911376
R^2: 0.055420717680097376

Support Vector Regression - Evaluation:
MSE: 14047.213878222294
MAE: 53.40774215579572
R^2: -0.24254073580713587
